This is a tutorial to use the flip package: https://github.com/corentinravoux/flip \
It is self-contained and can be used in google collab or on your environement \
All the data used are subsampled version of a simulation. \
The data size is small for the tutorial, do not use it for science case. \

In [1]:
%%capture
!pip install git+https://github.com/corentinravoux/flip

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from flip import utils, fitter, plot_utils, data_vector, __flip_dir_path__
from flip.covariance import covariance, contraction

flip_base = Path(__flip_dir_path__)
data_path = flip_base / "data"
plt.style.use(data_path / "style.mplstyle")

[ 000000.00 ]: 09-23 02:18  root            INFO     No pypower module detected, gridding with this method is unavailable
ERROR:2025-09-23 02:18:09,792:jax._src.xla_bridge:502: Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda13.initialize()
Traceback (most recent call last):
  File "/global/homes/r/ravouxco/.local/perlmutter/python-3.11/lib/python3.11/site-packages/jax/_src/xla_bridge.py", line 500, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/global/homes/r/ravouxco/.local/perlmutter/python-3.11/lib/python3.11/site-packages/jax_plugins/xla_cuda13/__init__.py", line 328, in initialize
    _check_cuda_versions(raise_on_first_error=True)
  File "/global/homes/r/ravouxco/.local/perlmutter/python-3.11/lib/python3.11/site-packages/jax_plugins/xla_cuda13/__init__.py", line 150, in _check_cuda_versions
    assert cuda_versions is not None
           ^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError
[ 000000.24 ]: 09-23 02:18  jax._src.xla_bridge ERROR    

Loading the data, located in the package itself



In [3]:
data_velocity = pd.read_parquet(data_path / "velocity_data.parquet")


data_velocity_true = {**data_velocity.rename(columns={'vpec': 'velocity'}).to_dict(orient='list'), 
                      "velocity_error": np.zeros(len(data_velocity["vpec"].values))}


# Create the data Vector
DataTrueVel = data_vector.DirectVel(data_velocity_true)

ktt, ptt = np.loadtxt(data_path / "power_spectrum_tt.txt")
kmt, pmt = np.loadtxt(data_path / "power_spectrum_mt.txt")
kmm, pmm = np.loadtxt(data_path / "power_spectrum_mm.txt")

sigmau_fiducial = 15

power_spectrum_dict = {"vv": [[ktt, ptt * utils.Du(ktt, sigmau_fiducial) ** 2]]}


# Fit true velocity interpolation sigma_u

In the model, there is a sigma_u parameter which can be interpreted to the smoothing of SNIa position due to peculiar velocities. \
To fit simultaneously this parameter, we interpolate the covariance matrix. 

In [7]:
sigmau_list = np.linspace(10.0, 20.0,20)[:, np.newaxis]
covariance_list = []
size_batch=10_000
number_worker=16


for sigu in sigmau_list:
    
    power_spectrum_dict = {"vv": [[ktt, ptt* utils.Du(ktt, sigu)**2]]}

    covariance_list.append(
        DataTrueVel.compute_covariance(
            "carreres23",
            power_spectrum_dict,
            size_batch=size_batch,
            number_worker=number_worker
        )
    )


[ 000077.42 ]: 09-23 02:19  root            INFO     Covariance matrix generated from flip with carreres23 model in 2.27e+00 seconds
[ 000079.69 ]: 09-23 02:19  root            INFO     Covariance matrix generated from flip with carreres23 model in 2.11e+00 seconds
[ 000081.89 ]: 09-23 02:19  root            INFO     Covariance matrix generated from flip with carreres23 model in 2.01e+00 seconds
[ 000084.20 ]: 09-23 02:19  root            INFO     Covariance matrix generated from flip with carreres23 model in 2.10e+00 seconds
[ 000086.40 ]: 09-23 02:19  root            INFO     Covariance matrix generated from flip with carreres23 model in 2.04e+00 seconds
[ 000088.66 ]: 09-23 02:19  root            INFO     Covariance matrix generated from flip with carreres23 model in 2.09e+00 seconds
[ 000091.05 ]: 09-23 02:19  root            INFO     Covariance matrix generated from flip with carreres23 model in 2.23e+00 seconds
[ 000093.70 ]: 09-23 02:19  root            INFO     Covariance matri

In [8]:
emulator_model = "gpmatrix"
emulator_parameter_names = ["sigu"]

covariance_gpmatrix = covariance.CovMatrix.init_from_emulator(
            emulator_model,
            "velocity",
            covariance_list,
            sigmau_list,
            emulator_parameter_names,
            verbose=True,
            test_emulator=False,
        )



[ 000142.26 ]: 09-23 02:20  GP              INFO     initializing Y
[ 000142.26 ]: 09-23 02:20  GP              INFO     initializing inference method
[ 000142.26 ]: 09-23 02:20  GP              INFO     adding kernel and likelihood as parameters


[ 000142.56 ]: 09-23 02:20  GP              INFO     initializing Y
[ 000142.56 ]: 09-23 02:20  GP              INFO     initializing inference method
[ 000142.56 ]: 09-23 02:20  GP              INFO     adding kernel and likelihood as parameters


[ 000283.42 ]: 09-23 02:22  root            INFO     Covariance matrix generated from emulator with gpmatrix model in 1.42e+02 seconds


In [9]:
parameter_dict = {"fs8": {"value": 0.4,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                  "sigv": {"value": 200,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                  "sigu": {"value": 15.5,
                          "limit_low" : 10.0,
                          "limit_up" : 20.0,
                          "fixed" : False,
                         },
                 }


prior_dict = {"sigu": {"type": "gaussian",
                       "mean" : 15.0,
                       "standard_deviation" : 5.0,},
             }

likelihood_type="multivariate_gaussian"
likelihood_properties = {"inversion_method": "cholesky",
                         "prior": prior_dict}



minuit_fitter = fitter.FitMinuit.init_from_covariance(
        covariance_gpmatrix,
        DataTrueVel,
        parameter_dict,
        likelihood_type=likelihood_type,
        likelihood_properties = likelihood_properties,
)

print('Free parameters:', minuit_fitter.likelihood.free_par)

minuit_fitter.run()


Free parameters: ['fs8', 'sigv']


[ 000314.95 ]: 09-23 02:23  root            INFO     ┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 1.374e+04                  │              Nfcn = 67               │
│ EDM = 1.98e-06 (Goal: 0.0001)    │           time = 12.5 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────

{'fs8': 0.36098297738210167,
 'sigv': 196.5163797427982,
 'sigu': 13.788453786685528}